In [2]:
# import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point

import h3
import math

import datatable as dt

In [144]:
%%time

ports0 = pd.read_csv('data/ports_baltic.csv')
ports0.rename( columns = { 
        'lat' : 'lat_port',
        'lon' : 'lon_port',
        'country_code' : 'iso2',
    }, inplace=True )

ports0['h3_5'] = ports0.apply(lambda row: h3.geo_to_h3( lat = row['lat_port'], lng = row['lon_port'], resolution = 5), axis = 1)
ports0['h3_5_hexring'] = ports0.apply(lambda row: h3.hex_ring( row['h3_5'], k = 1), axis = 1)
ports0 = ports0.explode('h3_5_hexring').reset_index()

dt.Frame(ports0)

CPU times: user 18.8 ms, sys: 0 ns, total: 18.8 ms
Wall time: 17.7 ms


,index,name,port_id,lat_port,lon_port,iso2,h3_5,h3_5_hexring
,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,0,VYSOTSKI,14928,60.616,28.566,ru,8511049bfffffff,8511048bfffffff
1,0,VYSOTSKI,14928,60.616,28.566,ru,8511049bfffffff,85110483fffffff
2,0,VYSOTSKI,14928,60.616,28.566,ru,8511049bfffffff,851104d7fffffff
3,0,VYSOTSKI,14928,60.616,28.566,ru,8511049bfffffff,85110493fffffff
4,0,VYSOTSKI,14928,60.616,28.566,ru,8511049bfffffff,85112267fffffff
5,0,VYSOTSKI,14928,60.616,28.566,ru,8511049bfffffff,8511226ffffffff
6,1,UST-LUGA,8214872,59.683,28.434,ru,85113103fffffff,8511310bfffffff
7,1,UST-LUGA,8214872,59.683,28.434,ru,85113103fffffff,85113113fffffff
8,1,UST-LUGA,8214872,59.683,28.434,ru,85113103fffffff,8511310ffffffff


In [145]:
ports2 = ports0.copy()
ports2['h3_5_hexring'] = ports2['h3_5']
ports2 = ports2.drop_duplicates(subset=['name','port_id','lat_port','lon_port','iso2','h3_5','h3_5_hexring'], keep = 'last').reset_index(drop = True) 
ports2 = ports2.sort_values(by=['name','port_id'], ascending=True)
ports = pd.concat([ports2, ports0]).sort_values(by=['name','port_id'], ascending=True)

ports['h3_5'] = ports['h3_5_hexring']
ports = ports[['name','port_id','lat_port','lon_port','iso2','h3_5']]
dt.Frame(ports)

,name,port_id,lat_port,lon_port,iso2,h3_5
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪
0,Agnesberg,2321,57.7856,12.0076,se,851f25abfffffff
1,Agnesberg,2321,57.7856,12.0076,se,851f2533fffffff
2,Agnesberg,2321,57.7856,12.0076,se,851f25affffffff
3,Agnesberg,2321,57.7856,12.0076,se,851f2517fffffff
4,Agnesberg,2321,57.7856,12.0076,se,851f25bbfffffff
5,Agnesberg,2321,57.7856,12.0076,se,851f25a3fffffff
6,Agnesberg,2321,57.7856,12.0076,se,851f2507fffffff
7,BERGKVARA,9238,56.3832,16.0831,se,851f28affffffff
8,BERGKVARA,9238,56.3832,16.0831,se,851f28a7fffffff


In [256]:
import h3.api.basic_int as h3_int

In [9]:
%%time

ports0 = pd.read_csv('data/ports_baltic.csv')
ports0.rename( columns = { 
        'lat' : 'lat_port',
        'lon' : 'lon_port',
        'country_code' : 'iso2',
    }, inplace=True )

ports0['h3_5'] = ports0.apply(lambda row: h3_int.geo_to_h3( lat = row['lat_port'], lng = row['lon_port'], resolution = 5), axis = 1)
ports0['h3_5_hexring'] = ports0.apply(lambda row: h3_int.hex_ring( row['h3_5'], k = 1), axis = 1)
ports0 = ports0.explode('h3_5_hexring').reset_index()

dt.Frame(ports0)

CPU times: user 16.8 ms, sys: 541 µs, total: 17.4 ms
Wall time: 15.9 ms


,index,name,port_id,lat_port,lon_port,iso2,h3_5,h3_5_hexring
,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,VYSOTSKI,14928,60.616,28.566,ru,599278134356869119,599280184130011135
1,0,VYSOTSKI,14928,60.616,28.566,ru,599278134356869119,599278150462996479
2,0,VYSOTSKI,14928,60.616,28.566,ru,599278134356869119,599280181982527487
3,0,VYSOTSKI,14928,60.616,28.566,ru,599278134356869119,599278132209385471
4,0,VYSOTSKI,14928,60.616,28.566,ru,599278134356869119,599278130061901823
5,0,VYSOTSKI,14928,60.616,28.566,ru,599278134356869119,599278127914418175
6,1,UST-LUGA,8214872,59.683,28.434,ru,599281185931132927,599281192373583871
7,1,UST-LUGA,8214872,59.683,28.434,ru,599281185931132927,599281189152358399
8,1,UST-LUGA,8214872,59.683,28.434,ru,599281185931132927,599281190226100223


In [10]:
ports2 = ports0.copy()
ports2['h3_5_hexring'] = ports2['h3_5']
ports2 = ports2.drop_duplicates(subset=['name','port_id','lat_port','lon_port','iso2','h3_5','h3_5_hexring'], keep = 'last').reset_index(drop = True) 
ports2 = ports2.sort_values(by=['name','port_id'], ascending=True)
ports = pd.concat([ports2, ports0]).sort_values(by=['name','port_id'], ascending=True)

ports['h3_5'] = ports['h3_5_hexring']
ports = ports[['name','port_id','lat_port','lon_port','iso2','h3_5']]
dt.Frame(ports)

,name,port_id,lat_port,lon_port,iso2,h3_5
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,Agnesberg,2321,57.7856,12.0076,se,599526696999190527
1,Agnesberg,2321,57.7856,12.0076,se,599526657270743039
2,Agnesberg,2321,57.7856,12.0076,se,599526664786935807
3,Agnesberg,2321,57.7856,12.0076,se,599526701294157823
4,Agnesberg,2321,57.7856,12.0076,se,599526694851706879
5,Agnesberg,2321,57.7856,12.0076,se,599526698072932351
6,Agnesberg,2321,57.7856,12.0076,se,599526652975775743
7,BERGKVARA,9238,56.3832,16.0831,se,599526904231362559
8,BERGKVARA,9238,56.3832,16.0831,se,599526872019107839


In [14]:
ports.to_csv('ports.csv')

In [15]:
ports

,name,port_id,lat_port,lon_port,iso2,h3_5
119,Agnesberg,2321,57.7856,12.0076,se,599526696999190527
714,Agnesberg,2321,57.7856,12.0076,se,599526657270743039
715,Agnesberg,2321,57.7856,12.0076,se,599526664786935807
716,Agnesberg,2321,57.7856,12.0076,se,599526701294157823
717,Agnesberg,2321,57.7856,12.0076,se,599526694851706879
...,...,...,...,...,...,...
697,ÖRNSKÖLDSVIK,939929,63.2830,18.7160,se,599136434795839487
698,ÖRNSKÖLDSVIK,939929,63.2830,18.7160,se,599131128363745279
699,ÖRNSKÖLDSVIK,939929,63.2830,18.7160,se,599131129437487103
700,ÖRNSKÖLDSVIK,939929,63.2830,18.7160,se,599136449828225023


In [17]:
ports['coordinate'] = ports[['lon_port','lat_port']].values.tolist()
ports

,name,port_id,lat_port,lon_port,iso2,h3_5,coordinate
119,Agnesberg,2321,57.7856,12.0076,se,599526696999190527,"[12.0076, 57.7856]"
714,Agnesberg,2321,57.7856,12.0076,se,599526657270743039,"[12.0076, 57.7856]"
715,Agnesberg,2321,57.7856,12.0076,se,599526664786935807,"[12.0076, 57.7856]"
716,Agnesberg,2321,57.7856,12.0076,se,599526701294157823,"[12.0076, 57.7856]"
717,Agnesberg,2321,57.7856,12.0076,se,599526694851706879,"[12.0076, 57.7856]"
...,...,...,...,...,...,...,...
697,ÖRNSKÖLDSVIK,939929,63.2830,18.7160,se,599136434795839487,"[18.716, 63.283]"
698,ÖRNSKÖLDSVIK,939929,63.2830,18.7160,se,599131128363745279,"[18.716, 63.283]"
699,ÖRNSKÖLDSVIK,939929,63.2830,18.7160,se,599131129437487103,"[18.716, 63.283]"
700,ÖRNSKÖLDSVIK,939929,63.2830,18.7160,se,599136449828225023,"[18.716, 63.283]"


In [33]:
#  df.groupby('a')['b'].apply(list)

ports4 = ports.groupby(['name', 'port_id', 'lat_port', 'lon_port', 'iso2'])['coordinate'].apply(list).reset_index(name='movement_coordinates')
ports4

,name,port_id,lat_port,lon_port,iso2,movement_coordinates
0,Agnesberg,2321,57.785600,12.007600,se,"[[12.0076, 57.7856], [12.0076, 57.7856], [12.0..."
1,BERGKVARA,9238,56.383200,16.083100,se,"[[16.0831, 56.3832], [16.0831, 56.3832], [16.0..."
2,BOHUS,8763428,57.824000,12.016124,se,"[[12.016124, 57.824], [12.016124, 57.824], [12..."
3,BROFJORDEN,4219,58.350000,11.416492,se,"[[11.416492, 58.35], [11.416492, 58.35], [11.4..."
4,BÅLSTA,4949,59.583300,17.504000,se,"[[17.504, 59.5833], [17.504, 59.5833], [17.504..."
...,...,...,...,...,...,...
147,YSTAD,130593,55.433324,13.830000,se,"[[13.83, 55.4333243], [13.83, 55.4333243], [13..."
148,ÅHUS,89412,55.933320,14.316325,se,"[[14.316325, 55.93332], [14.316325, 55.93332],..."
149,ÅMÅL,93251,59.052410,12.716421,se,"[[12.716421, 59.05241], [12.716421, 59.05241],..."
150,ÖREGRUND,4210,60.334210,18.459530,se,"[[18.45953, 60.33421], [18.45953, 60.33421], [..."


In [50]:
# ais_03_movement.groupby(['msisdn','stop_movement'])['timestamp'].transform('first')
ports['movement_coordinates'] = ports.groupby(['port_id'])['coordinate'].apply(list)
ports['lat_mean'] = ports.groupby(['iso2'])['lat_port'].transform(np.mean)
ports['lat_max'] = ports.groupby(['iso2'])['lat_port'].transform('max')
ports['lat_min'] = ports.groupby(['iso2'])['lat_port'].transform('min')
ports['lat_sum'] = ports.groupby(['iso2'])['lat_port'].transform('sum')
ports['lonlat'] = ports['lon_port'].map(str) + '-' + ports['lat_port'].map(str)
ports['lat_first'] = ports.groupby(['iso2'])['lat_port'].transform('first')
ports['lat_first_2'] = ports.groupby(['iso2'])['lat_port'].first()

ports

,name,port_id,lat_port,lon_port,iso2,h3_5,coordinate,movement_coordinates,lat_mean,lat_max,lat_min,lat_sum,lonlat,lat_first,lat_first_2
119,Agnesberg,2321,57.7856,12.0076,se,599526696999190527,"[12.0076, 57.7856]",NaN,59.062257,65.912,55.366686,34315.171408,12.0076-57.7856,57.7856,NaN
714,Agnesberg,2321,57.7856,12.0076,se,599526657270743039,"[12.0076, 57.7856]",NaN,59.062257,65.912,55.366686,34315.171408,12.0076-57.7856,57.7856,NaN
715,Agnesberg,2321,57.7856,12.0076,se,599526664786935807,"[12.0076, 57.7856]",NaN,59.062257,65.912,55.366686,34315.171408,12.0076-57.7856,57.7856,NaN
716,Agnesberg,2321,57.7856,12.0076,se,599526701294157823,"[12.0076, 57.7856]",NaN,59.062257,65.912,55.366686,34315.171408,12.0076-57.7856,57.7856,NaN
717,Agnesberg,2321,57.7856,12.0076,se,599526694851706879,"[12.0076, 57.7856]",NaN,59.062257,65.912,55.366686,34315.171408,12.0076-57.7856,57.7856,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,ÖRNSKÖLDSVIK,939929,63.2830,18.7160,se,599136434795839487,"[18.716, 63.283]",NaN,59.062257,65.912,55.366686,34315.171408,18.716-63.283,57.7856,NaN
698,ÖRNSKÖLDSVIK,939929,63.2830,18.7160,se,599131128363745279,"[18.716, 63.283]",NaN,59.062257,65.912,55.366686,34315.171408,18.716-63.283,57.7856,NaN
699,ÖRNSKÖLDSVIK,939929,63.2830,18.7160,se,599131129437487103,"[18.716, 63.283]",NaN,59.062257,65.912,55.366686,34315.171408,18.716-63.283,57.7856,NaN
700,ÖRNSKÖLDSVIK,939929,63.2830,18.7160,se,599136449828225023,"[18.716, 63.283]",NaN,59.062257,65.912,55.366686,34315.171408,18.716-63.283,57.7856,NaN


In [51]:
def haversine(lon1, lat1, lon2, lat2):                                              
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    a = np.sin((lat2 - lat1) / 2.0)**2 + (np.cos(lat1) * np.cos(lat2) * np.sin((lon2 - lon1) / 2.0)**2)
    distance = 6371 * 2 * np.arcsin(np.sqrt(a))
    return distance

def speed(distance, duration):
    time = duration / 3600
    speed = distance / time
    return speed

In [55]:
%%time
ports['distance_2'] = ports.apply(lambda row: haversine( 
                                  lon1 = row['lon_port'], lat1 = row['lat_port'],
                                  lon2 = row['lon_port'], lat2 = row['lat_port']
                                ), axis = 1
                             )

ports

CPU times: user 47.8 ms, sys: 3.19 ms, total: 51 ms
Wall time: 49.8 ms


,name,port_id,lat_port,lon_port,iso2,h3_5,coordinate,movement_coordinates,lat_mean,lat_max,lat_min,lat_sum,lonlat,lat_first,lat_first_2,distance,distance_2
119,Agnesberg,2321,57.7856,12.0076,se,599526696999190527,"[12.0076, 57.7856]",NaN,59.062257,65.912,55.366686,34315.171408,12.0076-57.7856,57.7856,NaN,0.0,0.0
714,Agnesberg,2321,57.7856,12.0076,se,599526657270743039,"[12.0076, 57.7856]",NaN,59.062257,65.912,55.366686,34315.171408,12.0076-57.7856,57.7856,NaN,0.0,0.0
715,Agnesberg,2321,57.7856,12.0076,se,599526664786935807,"[12.0076, 57.7856]",NaN,59.062257,65.912,55.366686,34315.171408,12.0076-57.7856,57.7856,NaN,0.0,0.0
716,Agnesberg,2321,57.7856,12.0076,se,599526701294157823,"[12.0076, 57.7856]",NaN,59.062257,65.912,55.366686,34315.171408,12.0076-57.7856,57.7856,NaN,0.0,0.0
717,Agnesberg,2321,57.7856,12.0076,se,599526694851706879,"[12.0076, 57.7856]",NaN,59.062257,65.912,55.366686,34315.171408,12.0076-57.7856,57.7856,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,ÖRNSKÖLDSVIK,939929,63.2830,18.7160,se,599136434795839487,"[18.716, 63.283]",NaN,59.062257,65.912,55.366686,34315.171408,18.716-63.283,57.7856,NaN,0.0,0.0
698,ÖRNSKÖLDSVIK,939929,63.2830,18.7160,se,599131128363745279,"[18.716, 63.283]",NaN,59.062257,65.912,55.366686,34315.171408,18.716-63.283,57.7856,NaN,0.0,0.0
699,ÖRNSKÖLDSVIK,939929,63.2830,18.7160,se,599131129437487103,"[18.716, 63.283]",NaN,59.062257,65.912,55.366686,34315.171408,18.716-63.283,57.7856,NaN,0.0,0.0
700,ÖRNSKÖLDSVIK,939929,63.2830,18.7160,se,599136449828225023,"[18.716, 63.283]",NaN,59.062257,65.912,55.366686,34315.171408,18.716-63.283,57.7856,NaN,0.0,0.0


In [53]:
%%time
ports['distance'] = haversine(ports['lon_port'], ports['lat_port'],ports['lon_port'], ports['lat_port'])
ports
# haversine_np(df['lon1'],df['lat1'],df['lon2'],df['lat2'])

CPU times: user 6.04 ms, sys: 885 µs, total: 6.93 ms
Wall time: 5.92 ms


,name,port_id,lat_port,lon_port,iso2,h3_5,coordinate,movement_coordinates,lat_mean,lat_max,lat_min,lat_sum,lonlat,lat_first,lat_first_2,distance
119,Agnesberg,2321,57.7856,12.0076,se,599526696999190527,"[12.0076, 57.7856]",NaN,59.062257,65.912,55.366686,34315.171408,12.0076-57.7856,57.7856,NaN,0.0
714,Agnesberg,2321,57.7856,12.0076,se,599526657270743039,"[12.0076, 57.7856]",NaN,59.062257,65.912,55.366686,34315.171408,12.0076-57.7856,57.7856,NaN,0.0
715,Agnesberg,2321,57.7856,12.0076,se,599526664786935807,"[12.0076, 57.7856]",NaN,59.062257,65.912,55.366686,34315.171408,12.0076-57.7856,57.7856,NaN,0.0
716,Agnesberg,2321,57.7856,12.0076,se,599526701294157823,"[12.0076, 57.7856]",NaN,59.062257,65.912,55.366686,34315.171408,12.0076-57.7856,57.7856,NaN,0.0
717,Agnesberg,2321,57.7856,12.0076,se,599526694851706879,"[12.0076, 57.7856]",NaN,59.062257,65.912,55.366686,34315.171408,12.0076-57.7856,57.7856,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,ÖRNSKÖLDSVIK,939929,63.2830,18.7160,se,599136434795839487,"[18.716, 63.283]",NaN,59.062257,65.912,55.366686,34315.171408,18.716-63.283,57.7856,NaN,0.0
698,ÖRNSKÖLDSVIK,939929,63.2830,18.7160,se,599131128363745279,"[18.716, 63.283]",NaN,59.062257,65.912,55.366686,34315.171408,18.716-63.283,57.7856,NaN,0.0
699,ÖRNSKÖLDSVIK,939929,63.2830,18.7160,se,599131129437487103,"[18.716, 63.283]",NaN,59.062257,65.912,55.366686,34315.171408,18.716-63.283,57.7856,NaN,0.0
700,ÖRNSKÖLDSVIK,939929,63.2830,18.7160,se,599136449828225023,"[18.716, 63.283]",NaN,59.062257,65.912,55.366686,34315.171408,18.716-63.283,57.7856,NaN,0.0


In [275]:
h3.geo_to_h3(60.143988, 20.177059, 1)

'8108bffffffffff'

In [276]:
h3_1_baltic = h3.hex_ring('8108bffffffffff', 1)
h3_1_baltic

{'81083ffffffffff',
 '8108fffffffffff',
 '8109bffffffffff',
 '81113ffffffffff',
 '811f3ffffffffff',
 '811f7ffffffffff'}

In [207]:
h3_1_int_baltic = h3_int.hex_ring(581118283558682623, 1)
h3_1_int_baltic

{581109487465660415,
 581122681605193727,
 581135875744727039,
 581267817140060159,
 581514107744681983,
 581518505791193087}

In [279]:
h3.geo_to_h3(43.143988, 34.177059, 1)

'812d3ffffffffff'

In [280]:
h3_1_black = h3.hex_ring('812d3ffffffffff', 1)
h3_1_black

{'811e7ffffffffff',
 '811efffffffffff',
 '812c3ffffffffff',
 '812d7ffffffffff',
 '812dbffffffffff',
 '813f7ffffffffff'}

In [127]:
%%time

ports_wfp = pd.read_csv('data/ports_wfp.csv')
ports_wfp.rename( columns = { 
        'lat' : 'lat_port',
        'lon' : 'lon_port',
        'country_code' : 'iso2',
    }, inplace=True )

ports_wfp['h3_1_int_baltic'] = ports_wfp.apply(lambda row: h3_int.geo_to_h3( lat = row['lat_port'], lng = row['lon_port'], resolution = 1), axis = 1)
# ports0['h3_1_int_hexring'] = ports0.apply(lambda row: h3_int.hex_ring( row['h3_5'], k = 1), axis = 1)
# ports0 = ports0.explode('h3_1_int_hexring').reset_index()

# ports0['h3_5'] = ports0.apply(lambda row: h3.geo_to_h3( lat = row['lat_port'], lng = row['lon_port'], resolution = 5), axis = 1)
# ports0['h3_5_hexring'] = ports0.apply(lambda row: h3.hex_ring( row['h3_5'], k = 1), axis = 1)
# ports0 = ports0.explode('h3_5_hexring').reset_index()

dt.Frame(ports_wfp)

CPU times: user 81.5 ms, sys: 2.94 ms, total: 84.4 ms
Wall time: 83.3 ms


,FID,portname,code,prttype,prtsize,status,maxdepth,maxlength,annualcapacitymt,humuse,…,gdb_geomattr_data,shape,h_index,iso2,h3_1_int_baltic
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,wld_trs_ports_wfp.14314,Watsi-Genge,NA,River,Very Small,Unknown,NA,NA,NA,Unknown,…,NA,POINT (20.629661913000007 -0.9456024360000015),846a917ffffffff,cd,582842317791035391
1,wld_trs_ports_wfp.14315,Charlotte (Skidegate),CASKI,Sea,Unknown,Open,NA,NA,NA,Unknown,…,NA,POINT (-132.009692535 53.247424029),8412947ffffffff,ca,581294205419126783
2,wld_trs_ports_wfp.14316,Homer,USHOI,Sea,Unknown,Open,NA,NA,NA,Unknown,…,NA,POINT (-151.415532437 59.60158521400001),840c51dffffffff,us,581184254256349183
3,wld_trs_ports_wfp.14317,Hartsdale,USHAS,Sea,Unknown,Open,NA,NA,NA,Unknown,…,NA,POINT (-74.00209774799998 40.70610420999998),842a107ffffffff,us,581707621791170559
4,wld_trs_ports_wfp.14318,Killingholme,GBKGH,Sea,Unknown,Unknown,NA,NA,NA,Unknown,…,NA,POINT (-0.2151185050000208 53.64560881900002),8419405ffffffff,gb,581412952674926591
5,wld_trs_ports_wfp.14319,Shoreham,GBSHO,Sea,Unknown,Unknown,NA,NA,NA,Unknown,…,NA,POINT (-0.2693394500000181 50.83231674299998),84194a7ffffffff,gb,581412952674926591
6,wld_trs_ports_wfp.14320,Southend,GBSND,Sea,Unknown,Unknown,NA,NA,NA,Unknown,…,NA,POINT (0.7154950909999798 51.53414750500002),84194e3ffffffff,gb,581412952674926591
7,wld_trs_ports_wfp.14321,Santa Fe,ARSFN,River,Medium,Unknown,NA,NA,NA,Unknown,…,NA,POINT (-60.697123858 -31.650138447000018),84b24d3ffffffff,ar,584104557139722239
8,wld_trs_ports_wfp.14322,Punta De Palmas,VEPLV,Sea,Small,Unknown,NA,NA,NA,Unknown,…,NA,POINT (-71.634715035 10.440904627999998),84662c9ffffffff,ve,582763152953835519


In [128]:
h3_1_int = pd.read_csv('data/h3_1_int.csv')
h3_1_int

,area,h3_1_int_baltic
0,baltic,581118283558682623
1,baltic,581109487465660415
2,baltic,581122681605193727
3,baltic,581135875744727039
4,baltic,581267817140060159
5,baltic,581514107744681983
6,baltic,58151850579119308
7,black,581760398349303807
8,black,581500913605148671
9,black,581509709698170879


In [129]:
ports_wfp_baltic = ports_wfp.join(h3_1_int.set_index('h3_1_int_baltic'), on='h3_1_int_baltic', how='left')
ports_wfp_baltic

,FID,portname,code,prttype,prtsize,status,maxdepth,maxlength,annualcapacitymt,humuse,...,source,createdate,updatedate,geonameid,gdb_geomattr_data,shape,h_index,iso2,h3_1_int_baltic,area
0,wld_trs_ports_wfp.14314,Watsi-Genge,NaN,River,Very Small,Unknown,NaN,NaN,NaN,Unknown,...,NaN,2021-02-24T11:52:47.493,2021-02-24T11:52:47.493,204280,NaN,POINT (20.629661913000007 -0.9456024360000015),846a917ffffffff,cd,582842317791035391,NaN
1,wld_trs_ports_wfp.14315,Charlotte (Skidegate),CASKI,Sea,Unknown,Open,NaN,NaN,NaN,Unknown,...,NaN,2021-02-24T11:52:47.493,2021-02-24T11:52:47.493,6148858,NaN,POINT (-132.009692535 53.247424029),8412947ffffffff,ca,581294205419126783,NaN
2,wld_trs_ports_wfp.14316,Homer,USHOI,Sea,Unknown,Open,NaN,NaN,NaN,Unknown,...,NaN,2021-02-24T11:52:47.493,2021-02-24T11:52:47.493,5864145,NaN,POINT (-151.415532437 59.60158521400001),840c51dffffffff,us,581184254256349183,NaN
3,wld_trs_ports_wfp.14317,Hartsdale,USHAS,Sea,Unknown,Open,NaN,NaN,NaN,Unknown,...,NaN,2021-02-24T11:52:47.493,2021-02-24T11:52:47.493,5120141,NaN,POINT (-74.00209774799998 40.70610420999998),842a107ffffffff,us,581707621791170559,NaN
4,wld_trs_ports_wfp.14318,Killingholme,GBKGH,Sea,Unknown,Unknown,NaN,NaN,NaN,Unknown,...,NaN,2021-02-24T11:52:47.493,2021-02-24T11:52:47.493,2641323,NaN,POINT (-0.2151185050000208 53.64560881900002),8419405ffffffff,gb,581412952674926591,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3576,wld_trs_ports_wfp.18294,Saint-Louis du Sud,NaN,NaN,NaN,Open,NaN,NaN,NaN,NaN,...,WFP Supply Chain,2022-10-07T00:00:00,2022-10-08T06:45:00.273,3717596,NaN,POINT (-73.54316612600002 18.257311448999985),8467263ffffffff,ht,582780745139879935,NaN
3577,wld_trs_ports_wfp.17889,Saida Commercial Port,NaN,sea,small,NaN,NaN,NaN,NaN,NaN,...,WFP CO,2021-12-23T12:18:13.643,2021-12-24T07:04:28.843,999999999,NaN,POINT (35.36527382999998 33.556561518000024),842db19ffffffff,lb,581769194442326015,black
3578,wld_trs_ports_wfp.17890,Sour Commercial Port,NaN,sea,small,NaN,NaN,NaN,NaN,NaN,...,WFP CO,2021-12-23T12:18:13.643,2021-12-24T07:09:13.126,999999999,NaN,POINT (35.197958906 33.27381918100002),842db19ffffffff,lb,581769194442326015,black
3579,wld_trs_ports_wfp.17891,Aamshite Fuel Oil Port,NaN,sea,small,NaN,NaN,NaN,NaN,NaN,...,WFP CO,2021-12-23T12:18:13.643,2021-12-24T07:13:58.906,999999999,NaN,POINT (35.630441184000006 34.15433912200001),842da27ffffffff,lb,581769194442326015,black


In [137]:
ports_wfp_baltic[ports_wfp_baltic['area'] == 'black']

,FID,portname,code,prttype,prtsize,status,maxdepth,maxlength,annualcapacitymt,humuse,...,source,createdate,updatedate,geonameid,gdb_geomattr_data,shape,h_index,iso2,h3_1_int_baltic,area
18,wld_trs_ports_wfp.14332,Eleusis,GREEU,Sea,Unknown,Unknown,NaN,NaN,NaN,Unknown,...,NaN,2021-02-24T11:52:47.493,2021-02-24T11:52:47.493,264371,NaN,POINT (23.51073838299999 38.042739654),841eda7ffffffff,gr,581509709698170879,black
19,wld_trs_ports_wfp.14333,Syros (Syra),GRJSY,Sea,Unknown,Unknown,NaN,NaN,NaN,Unknown,...,NaN,2021-02-24T11:52:47.493,2021-02-24T11:52:47.493,999999999,NaN,POINT (24.94031943599998 37.43767341),843f75dffffffff,gr,582081455744614399,black
23,wld_trs_ports_wfp.14337,Sulina (Port Et Zone Franche),ROSUL,Sea,Unknown,Unknown,NaN,NaN,NaN,Unknown,...,NaN,2021-02-24T11:52:47.493,2021-02-24T11:52:47.493,665790,NaN,POINT (29.662183481 45.158485703),841e5a9ffffffff,ro,581500913605148671,black
68,wld_trs_ports_wfp.14383,Sidi Kerir,EGSKT,Sea,Very Small,Open,NaN,NaN,NaN,Unknown,...,NaN,2021-02-24T11:52:47.493,2021-02-24T11:52:47.493,353407,NaN,POINT (29.670176771 31.05358998600002),843f5bbffffffff,eg,582081455744614399,black
75,wld_trs_ports_wfp.14392,Kalecik,CYKAL,Sea,Unknown,Open,NaN,NaN,NaN,Unknown,...,NaN,2021-02-24T11:52:47.493,2021-02-24T11:52:47.493,999999999,NaN,POINT (33.99318967800002 35.32798396999999),842da47ffffffff,cy,581769194442326015,black
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3573,wld_trs_ports_wfp.17888,Dbaiye 2 Port,NaN,sea,small,NaN,NaN,NaN,NaN,NaN,...,WFP CO,2021-12-23T12:18:13.643,2021-12-24T06:59:42.951,999999999,NaN,POINT (35.58868302000002 33.94409927100003),842da25ffffffff,lb,581769194442326015,black
3577,wld_trs_ports_wfp.17889,Saida Commercial Port,NaN,sea,small,NaN,NaN,NaN,NaN,NaN,...,WFP CO,2021-12-23T12:18:13.643,2021-12-24T07:04:28.843,999999999,NaN,POINT (35.36527382999998 33.556561518000024),842db19ffffffff,lb,581769194442326015,black
3578,wld_trs_ports_wfp.17890,Sour Commercial Port,NaN,sea,small,NaN,NaN,NaN,NaN,NaN,...,WFP CO,2021-12-23T12:18:13.643,2021-12-24T07:09:13.126,999999999,NaN,POINT (35.197958906 33.27381918100002),842db19ffffffff,lb,581769194442326015,black
3579,wld_trs_ports_wfp.17891,Aamshite Fuel Oil Port,NaN,sea,small,NaN,NaN,NaN,NaN,NaN,...,WFP CO,2021-12-23T12:18:13.643,2021-12-24T07:13:58.906,999999999,NaN,POINT (35.630441184000006 34.15433912200001),842da27ffffffff,lb,581769194442326015,black


In [185]:
ports_wfp_black0 = ports_wfp_baltic[['FID','portname','lat_port','lon_port','iso2','h3_1_int_baltic','area']][ports_wfp_baltic['area'] == 'black']
ports_wfp_black0

,FID,portname,lat_port,lon_port,iso2,h3_1_int_baltic,area
18,wld_trs_ports_wfp.14332,Eleusis,38.04274,23.51074,gr,581509709698170879,black
19,wld_trs_ports_wfp.14333,Syros (Syra),37.43767,24.94032,gr,582081455744614399,black
23,wld_trs_ports_wfp.14337,Sulina (Port Et Zone Franche),45.15849,29.66218,ro,581500913605148671,black
68,wld_trs_ports_wfp.14383,Sidi Kerir,31.05359,29.67018,eg,582081455744614399,black
75,wld_trs_ports_wfp.14392,Kalecik,35.32798,33.99319,cy,581769194442326015,black
...,...,...,...,...,...,...,...
3573,wld_trs_ports_wfp.17888,Dbaiye 2 Port,33.94410,35.58868,lb,581769194442326015,black
3577,wld_trs_ports_wfp.17889,Saida Commercial Port,33.55656,35.36527,lb,581769194442326015,black
3578,wld_trs_ports_wfp.17890,Sour Commercial Port,33.27382,35.19796,lb,581769194442326015,black
3579,wld_trs_ports_wfp.17891,Aamshite Fuel Oil Port,34.15434,35.63044,lb,581769194442326015,black


In [186]:
ports_wfp_black0['port_id'] = ports_wfp_black0['FID'].str.slice(14)

ports_wfp_black0 = ports_wfp_black0[['portname','port_id','lat_port','lon_port','iso2','area']]
ports_wfp_black.rename( columns = { 'portname' : 'name', }, inplace=True )

ports_wfp_black['h3_5'] = ports_wfp_black.apply(lambda row: h3.geo_to_h3( lat = row['lat_port'], lng = row['lon_port'], resolution = 5), axis = 1)
ports_wfp_black['h3_5_hexring'] = ports_wfp_black.apply(lambda row: h3.hex_ring( row['h3_5'], k = 1), axis = 1)
ports_wfp_black = ports_wfp_black.explode('h3_5_hexring').reset_index()

ports_wfp_black

,level_0,index,name,port_id,lat_port,lon_port,iso2,area,h3_5,h3_5_hexring
0,0,18,Eleusis,wfp.14332,38.04274,23.51074,gr,black,851eda73fffffff,851eda7bfffffff
1,0,18,Eleusis,wfp.14332,38.04274,23.51074,gr,black,851eda73fffffff,851eda0ffffffff
2,0,18,Eleusis,wfp.14332,38.04274,23.51074,gr,black,851eda73fffffff,851eda0bfffffff
3,0,18,Eleusis,wfp.14332,38.04274,23.51074,gr,black,851eda73fffffff,851eda47fffffff
4,0,18,Eleusis,wfp.14332,38.04274,23.51074,gr,black,851eda73fffffff,851eda63fffffff
...,...,...,...,...,...,...,...,...,...,...
7987,1331,3580,Chekka Fuel Oil Port,wfp.17892,34.32934,35.72545,lb,black,852da277fffffff,852da20ffffffff
7988,1331,3580,Chekka Fuel Oil Port,wfp.17892,34.32934,35.72545,lb,black,852da277fffffff,852da267fffffff
7989,1331,3580,Chekka Fuel Oil Port,wfp.17892,34.32934,35.72545,lb,black,852da277fffffff,852da273fffffff
7990,1331,3580,Chekka Fuel Oil Port,wfp.17892,34.32934,35.72545,lb,black,852da277fffffff,852da22bfffffff


In [193]:
ports_wfp_black2 = ports_wfp_black.copy()

ports_wfp_black2['h3_5_hexring'] = ports_wfp_black2['h3_5']
ports_wfp_black2 = ports_wfp_black2.drop_duplicates(subset=['name','port_id','lat_port','lon_port','iso2','area','h3_5','h3_5_hexring'], keep = 'last').reset_index(drop = True) 
ports_wfp_black2 = ports_wfp_black2.sort_values(by=['name','port_id'], ascending=True)

ports_black = pd.concat([ports_wfp_black2, ports_wfp_black]).sort_values(by=['name','port_id'], ascending=True)
ports_black['h3_5'] = ports_black['h3_5_hexring']
ports_black = ports_black[['name','port_id','lat_port','lon_port','iso2','area','h3_5']]
ports_black = ports_black.drop_duplicates(subset=['name','port_id','lat_port','lon_port','iso2','area','h3_5'], keep = 'last').reset_index(drop = True) 

ports_black

,name,port_id,lat_port,lon_port,iso2,area,h3_5
0,Aamshite,wfp.16923,34.15214,35.62828,lb,black,852da273fffffff
1,Aamshite,wfp.16923,34.15214,35.62828,lb,black,852da20bfffffff
2,Aamshite,wfp.16923,34.15214,35.62828,lb,black,852da20ffffffff
3,Aamshite,wfp.16923,34.15214,35.62828,lb,black,852da27bfffffff
4,Aamshite,wfp.16923,34.15214,35.62828,lb,black,852da277fffffff
...,...,...,...,...,...,...,...
1549,Zouq,wfp.17240,33.96980,35.60259,lb,black,852da26bfffffff
1550,Zouq,wfp.17240,33.96980,35.60259,lb,black,852db1b7fffffff
1551,Zouq,wfp.17240,33.96980,35.60259,lb,black,852da273fffffff
1552,Zouq,wfp.17240,33.96980,35.60259,lb,black,852da247fffffff


In [192]:
ports['area'] = 'baltic'
ports = ports[['name','port_id','lat_port','lon_port','iso2','area','h3_5']]
ports

,name,port_id,lat_port,lon_port,iso2,area,h3_5
119,Agnesberg,2321,57.7856,12.0076,se,baltic,851f25abfffffff
714,Agnesberg,2321,57.7856,12.0076,se,baltic,851f2533fffffff
715,Agnesberg,2321,57.7856,12.0076,se,baltic,851f25affffffff
716,Agnesberg,2321,57.7856,12.0076,se,baltic,851f2517fffffff
717,Agnesberg,2321,57.7856,12.0076,se,baltic,851f25bbfffffff
...,...,...,...,...,...,...,...
697,ÖRNSKÖLDSVIK,939929,63.2830,18.7160,se,baltic,8508a963fffffff
698,ÖRNSKÖLDSVIK,939929,63.2830,18.7160,se,baltic,8508f69bfffffff
699,ÖRNSKÖLDSVIK,939929,63.2830,18.7160,se,baltic,8508f6d7fffffff
700,ÖRNSKÖLDSVIK,939929,63.2830,18.7160,se,baltic,8508a96bfffffff


In [194]:
ports_ = pd.concat([ports, ports_black])
ports_

,name,port_id,lat_port,lon_port,iso2,area,h3_5
119,Agnesberg,2321,57.7856,12.00760,se,baltic,851f25abfffffff
714,Agnesberg,2321,57.7856,12.00760,se,baltic,851f2533fffffff
715,Agnesberg,2321,57.7856,12.00760,se,baltic,851f25affffffff
716,Agnesberg,2321,57.7856,12.00760,se,baltic,851f2517fffffff
717,Agnesberg,2321,57.7856,12.00760,se,baltic,851f25bbfffffff
...,...,...,...,...,...,...,...
1549,Zouq,wfp.17240,33.9698,35.60259,lb,black,852da26bfffffff
1550,Zouq,wfp.17240,33.9698,35.60259,lb,black,852db1b7fffffff
1551,Zouq,wfp.17240,33.9698,35.60259,lb,black,852da273fffffff
1552,Zouq,wfp.17240,33.9698,35.60259,lb,black,852da247fffffff


In [195]:
ports_['h3_5_int'] = ports_.apply(lambda row: h3_int.geo_to_h3( lat = row['lat_port'], lng = row['lon_port'], resolution = 5), axis = 1)
ports_

,name,port_id,lat_port,lon_port,iso2,area,h3_5,h3_5_int
119,Agnesberg,2321,57.7856,12.00760,se,baltic,851f25abfffffff,599526696999190527
714,Agnesberg,2321,57.7856,12.00760,se,baltic,851f2533fffffff,599526696999190527
715,Agnesberg,2321,57.7856,12.00760,se,baltic,851f25affffffff,599526696999190527
716,Agnesberg,2321,57.7856,12.00760,se,baltic,851f2517fffffff,599526696999190527
717,Agnesberg,2321,57.7856,12.00760,se,baltic,851f25bbfffffff,599526696999190527
...,...,...,...,...,...,...,...,...
1549,Zouq,wfp.17240,33.9698,35.60259,lb,black,852da26bfffffff,599781564653502463
1550,Zouq,wfp.17240,33.9698,35.60259,lb,black,852db1b7fffffff,599781564653502463
1551,Zouq,wfp.17240,33.9698,35.60259,lb,black,852da273fffffff,599781564653502463
1552,Zouq,wfp.17240,33.9698,35.60259,lb,black,852da247fffffff,599781564653502463


In [196]:
ports_.to_csv('ports_.csv')

In [198]:
ports__ = ports_.groupby([
                'name','port_id','lat_port','lon_port',
                'iso2','area'])['h3_5'].apply(list).reset_index(name='h3_5_hexring')

ports__

,name,port_id,lat_port,lon_port,iso2,area,h3_5_hexring
0,Aamshite,wfp.16923,34.15214,35.628280,lb,black,"[852da273fffffff, 852da20bfffffff, 852da20ffff..."
1,Aamshite Fuel Oil Port,wfp.17891,34.15434,35.630440,lb,black,"[852da273fffffff, 852da20bfffffff, 852da20ffff..."
2,Abu Qir,wfp.15978,31.32275,30.070860,eg,black,"[853f596bfffffff, 853f5963fffffff, 853f5ba7fff..."
3,Achladi,wfp.17639,38.89103,22.811340,gr,black,"[851ede63fffffff, 851ede6bfffffff, 851ede77fff..."
4,Agio Theodoroi,wfp.17321,37.92756,23.144540,gr,black,"[851eda4ffffffff, 851eda43fffffff, 851eda7bfff..."
...,...,...,...,...,...,...,...
369,Zouq,wfp.17240,33.96980,35.602590,lb,black,"[852da27bfffffff, 852da24ffffffff, 852da26bfff..."
370,ÅHUS,89412,55.93332,14.316325,se,baltic,"[851f2a93fffffff, 851f2a83fffffff, 851f2e2ffff..."
371,ÅMÅL,93251,59.05241,12.716421,se,baltic,"[8508b5d7fffffff, 8508b5c7fffffff, 8508b5d3fff..."
372,ÖREGRUND,4210,60.33421,18.459530,se,baltic,"[85088007fffffff, 8508800ffffffff, 8508803bfff..."


In [199]:
ports__.to_csv('ports__.csv')

In [208]:
h3_int.h3_get_resolution(599523168683556863)

5

In [250]:
%%time

ports0 = pd.read_csv('ports__.csv')
ports0.rename( columns = { 
        'lat' : 'lat_port',
        'lon' : 'lon_port',
        'country_code' : 'iso2',
    }, inplace=True )


# ports0['h3_5'] = ports0.apply(lambda row: h3.geo_to_h3( lat = row['lat_port'], lng = row['lon_port'], resolution = 5), axis = 1)
# ports0['h3_5_hexring'] = ports0.apply(lambda row: h3.hex_ring( row['h3_5'], k = 1), axis = 1)
# ports0 = ports0.explode('h3_5_hexring').reset_index()

ports0['h3_5_int'] = ports0.apply(lambda row: h3_int.geo_to_h3( lat = row['lat_port'], lng = row['lon_port'], resolution = 5), axis = 1)
ports0['h3_5_int_hexring'] = ports0.apply(lambda row: h3_int.hex_ring( row['h3_5_int'], k = 1), axis = 1)
ports0 = ports0.explode('h3_5_int_hexring').reset_index()

dt.Frame(ports0)

CPU times: user 26.1 ms, sys: 207 µs, total: 26.3 ms
Wall time: 25 ms


,index,Unnamed: 0,name,port_id,lat_port,lon_port,iso2,area,h3_5_hexring,h3_5_int,h3_5_int_hexring
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,0,Aamshite,wfp.16923,34.1521,35.6283,lb,black,"['852da273fffffff', '852da20bfffffff', '852da20fff…",599781562506018815,599781563579760639
1,0,0,Aamshite,wfp.16923,34.1521,35.6283,lb,black,"['852da273fffffff', '852da20bfffffff', '852da20fff…",599781562506018815,599781564653502463
2,0,0,Aamshite,wfp.16923,34.1521,35.6283,lb,black,"['852da273fffffff', '852da20bfffffff', '852da20fff…",599781562506018815,599781534588731391
3,0,0,Aamshite,wfp.16923,34.1521,35.6283,lb,black,"['852da273fffffff', '852da20bfffffff', '852da20fff…",599781562506018815,599781535662473215
4,0,0,Aamshite,wfp.16923,34.1521,35.6283,lb,black,"['852da273fffffff', '852da20bfffffff', '852da20fff…",599781562506018815,599781550694858751
5,0,0,Aamshite,wfp.16923,34.1521,35.6283,lb,black,"['852da273fffffff', '852da20bfffffff', '852da20fff…",599781562506018815,599781558211051519
6,1,1,Aamshite Fuel Oil Port,wfp.17891,34.1543,35.6304,lb,black,"['852da273fffffff', '852da20bfffffff', '852da20fff…",599781562506018815,599781563579760639
7,1,1,Aamshite Fuel Oil Port,wfp.17891,34.1543,35.6304,lb,black,"['852da273fffffff', '852da20bfffffff', '852da20fff…",599781562506018815,599781564653502463
8,1,1,Aamshite Fuel Oil Port,wfp.17891,34.1543,35.6304,lb,black,"['852da273fffffff', '852da20bfffffff', '852da20fff…",599781562506018815,599781534588731391


In [251]:
ports2 = ports0.copy()
ports2['h3_5_int_hexring'] = ports2['h3_5_int']
ports2 = ports2.drop_duplicates(subset=['name','port_id','lat_port','lon_port','iso2','area','h3_5_int','h3_5_int_hexring'], keep = 'last').reset_index(drop = True) 
ports2 = ports2.sort_values(by=['name','port_id'], ascending=True)
ports = pd.concat([ports2, ports0]).sort_values(by=['name','port_id'], ascending=True)

ports['h3_5_int'] = ports['h3_5_int_hexring']
ports = ports[['name','port_id','lat_port','lon_port','iso2','area','h3_5_int']]
# ports = ports.drop_duplicates(subset=['name','port_id','lat_port','lon_port','iso2','area'], keep = 'last').reset_index(drop = True) 
# ports = ports.sort_values(by=['name'], ascending=True)
dt.Frame(ports)

,name,port_id,lat_port,lon_port,iso2,area,h3_5_int
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,Aamshite,wfp.16923,34.1521,35.6283,lb,black,599781562506018815
1,Aamshite,wfp.16923,34.1521,35.6283,lb,black,599781563579760639
2,Aamshite,wfp.16923,34.1521,35.6283,lb,black,599781564653502463
3,Aamshite,wfp.16923,34.1521,35.6283,lb,black,599781534588731391
4,Aamshite,wfp.16923,34.1521,35.6283,lb,black,599781535662473215
5,Aamshite,wfp.16923,34.1521,35.6283,lb,black,599781550694858751
6,Aamshite,wfp.16923,34.1521,35.6283,lb,black,599781558211051519
7,Aamshite Fuel Oil Port,wfp.17891,34.1543,35.6304,lb,black,599781562506018815
8,Aamshite Fuel Oil Port,wfp.17891,34.1543,35.6304,lb,black,599781563579760639


In [255]:
ports.to_csv('ports_.csv')

In [258]:
%%time

# CPU times: user 170 ms, sys: 7.87 ms, total: 178 ms
# Wall time: 621 ms

url = 'https://raw.githubusercontent.com/datacerdas/know-ais/main/platform/src/data/h3_1_int.csv'
ports_1 = pd.read_csv(url, error_bad_lines=False)

ports_1
# ports_1 = spark.createDataFrame(ports_1) 
# ports_1.show()

CPU times: user 11.8 ms, sys: 2.22 ms, total: 14 ms
Wall time: 63.8 ms


<timed exec>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.




,area,h3_1_int_baltic
0,baltic,581118283558682623
1,baltic,581109487465660415
2,baltic,581122681605193727
3,baltic,581135875744727039
4,baltic,581267817140060159
5,baltic,581514107744681983
6,baltic,581518505791193087
7,black,581760398349303807
8,black,581500913605148671
9,black,581509709698170879


In [ ]:
h3_1_int_baltic = h3_int.hex_ring(581118283558682623, 1)
h3_1_int_baltic